In [10]:
from pathlib import Path

class args:
    seed = 41
    epochs=1000
    batch_size= 32
    img_height = 224
    img_width = 224
    ffn_size = 512
    workers= 1
    learning_rate_weights=0.2
    learning_rate_biases=0.0048
    weight_decay=1e-6
    lambd=0.0051
    projector=[ffn_size*16]*3
    print_freq=100
    checkpoint_dir=Path('../checkpoint/')
    data_dir = "../../dataset/FULL_DATASET/FIXATION_DATASET"
    datasets = ['SALICON', 'EMOD', 'FIWI', 'MITLowRes', 'OSIE', 'SIENA12', 'TORONTO', 'VIU']
    transform = dict(
        img_size = (img_height, img_width),
        noise = 0,
        drop = 0,
        reversal = 0,
        rotation = 0,
    )

args = args()

In [31]:
import json
import torch
from PIL import Image
from random import sample
import numpy as np
import torch.utils.data as data
import matplotlib.pyplot as plt
from torch.nn.functional import pad
import os

def read_dataset(anno_dir, datasets, val_ratio = 0.2):
    """
    We split each dataset in datasets list into train and val
    """
    train_anno = {'fixation': [],
                  'duration': [],
                  'img_id': [],
                  'dva': [],
                  'img_size': []}
    
    val_anno = {'fixation': [],
                'duration': [],
                'img_id': [],
                'dva': [],
                'img_size': []}
    
    ret_anno = (train_anno, val_anno)
    
    for data in datasets:
        anno = {}
        with open(f"{anno_dir}/ANNOTATIONS/{data}.json", 'r') as f:
            anno = json.load(f)
        ims = list(anno.keys())
        val_len = int(val_ratio * len(ims))
        val_ims = sample(ims, val_len)
        train_ims = list(set(ims) - set(val_ims))
        
        for i, g in enumerate((train_ims, val_ims)):
            for im in g:
                num = len(anno[im]['fixations'])
                ret_anno[i]['fixation'].extend(anno[im]['fixations'])
                if 'durations' in anno[im]:
                    ret_anno[i]['duration'].extend(anno[im]['durations'])
                else:
                    ret_anno[i]['duration'].extend([[0]]*num)
                    
                ret_anno[i]['img_id'].extend([f"{anno_dir}/STIMULI/{data}/{im}.jpg"]*num)
                if 'dva' in anno[im]:
                    ret_anno[i]['dva'].extend([anno[im]['dva']]*num)
                else:
                    ret_anno[i]['dva'].extend([None]*num)
                ret_anno[i]['img_size'].extend([anno[im]['img_size']]*num)
    
    return ret_anno

class FixationDataset(data.Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
        
    def __getitem__(self, index):
        assert os.exists(self.data['img_id'][index])
        fix = np.array(self.data['fixation'][index].copy())
        dur = np.array(self.data['duration'][index].copy())
        dva = self.data['dva'][index]
        
        return None
    
    def __len__(self):
        return len(self.data['img_id'])
    
    def collate_function(self, batch):
        v1_b, v2_b = zip(*batch)
        return self.pad_batch(v1_b), self.pad_batch(v2_b)

    def pad_batch(self, batch):
        img_b, fix_b, dur_b = zip(*batch)
        fix_b, dur_b = list(fix_b), list(dur_b)
        fix_lens = [len(fix) for fix in fix_b]
        max_fix_len = max(fix_lens)

        for i, fix in enumerate(fix_b):
            fix_b[i] = pad(fix, (0,0, 0, max_fix_len - len(fix)), "constant", 1e7)
            dur_b[i] = pad(dur_b[i], (0,max_fix_len - len(dur_b[i])), "constant", 1e7)

        return (torch.stack(img_b), torch.stack(fix_b), torch.stack(dur_b))

In [32]:
anno, _ = read_dataset(args.data_dir, args.datasets, val_ratio=0.0)
# dataset = FixationDataset(anno)

In [33]:
len(anno['fixation']), len(anno['duration'])

(924584, 924584)

In [34]:
for idx in range(len(anno['fixation'])):
    img_id = anno['img_id'][idx]
    fix = anno['fixation'][idx]
    dur = anno['duration'][idx]
    
    if not isinstance(dur, list): print(idx, img_id,fix,dur)

In [21]:
anno[908074]

KeyError: 908074